In [ ]:
import pandas as pd
import numpy as np

from elote import EloCompetitor, LambdaArena

import matplotlib.pyplot as plt

import random
import math

In [ ]:
# Load in full matches and wrestlers dataframes
full_matches = pd.read_csv('MATCHES.csv').drop(columns="Unnamed: 0")
full_wrestlers = pd.read_csv('WRESTLERS.csv').drop(columns="Unnamed: 0")

In [ ]:
full_matches.shape # number of matches, recorded match variables

In [ ]:
full_matches.columns

In [ ]:
full_wrestlers.shape # number of wrestlers, recorded wrestler variables

In [ ]:
full_wrestlers.columns

In [ ]:
# Drop matches decided by useless wincons like forfeit or bye (ASSUMPTION: these wincons don't imply wrestler ability)
bad_wins = ['Forfeit','Injury Default','Medical Forfeit','Bye','Disqualified','Default','No Contest']
win_filter = [win not in bad_wins for win in full_matches["Victory Type (L)"]]
MATCHES = full_matches.loc[win_filter].drop_duplicates().reset_index(drop=True) # dedupe seems to work now
MATCHES.sort_values(by='Event Date',ascending=True,inplace=True,ignore_index=True) # Make sure matches are sorted by date
# Go back and check if dedupe is removing more than it should -- maybe not distinguishing multiple bouts on same day
MATCHES.shape

In [ ]:
# Copy infoscrape function from Wrestling Tables notebook

def infoscrape(fullname,df):
    '''infoscrape receives full name of wrestler and matches dataframe
    and collects wrestler info from dataset'''

    # Initialize values of interest
    weight_class = 0
    wins = 0
    losses = 0
    matches = 0
    school = ''
    school_code = ''
    first_name = ''
    last_name = ''
    
    # Find observations corresponding to wrestler name
    win_id = df['Winner Full Name'] == fullname
    loss_id = df['Loser Full Name'] == fullname
    winning_matches = df.loc[win_id,:]
    losing_matches = df.loc[loss_id,:]
    
    # Split full name
    first_name, last_name = fullname.split(' ',1)
    
    # Counting stats (should check if names show in correct columns for forfeits, byes, etc.)
    wins = sum(win_id)
    losses = sum(loss_id)
    matches = wins+losses
    
    # Extract weight class, school, etc.
    win_weight = winning_matches['Weight Class'].unique()
    loss_weight = losing_matches['Weight Class'].unique()
    
    if win_weight.size > 0: # Avoiding 'if win_weight:' because it gives truth amibiguity warning
        weight_class = int(win_weight[0])
    else: # !!!Still need to add consideration for multiple weight classes!!!
        weight_class = int(loss_weight[0])
        
    win_school = winning_matches['Winner School (L)'].unique()
    win_school_code = winning_matches['Winner School (S)'].unique()
    loss_school = losing_matches['Loser School (L)'].unique()
    loss_school_code = losing_matches['Loser School (S)'].unique()
    
    if win_school.size > 0: # Avoiding 'if win_school:' because it gives truth amibiguity warning
        school = win_school[0]
        school_code = win_school_code[0]
    else: 
        school = loss_school[0]
        school_code = loss_school_code[0]
        
    # Return list of extracted data 
    return({'First Name':first_name,'Last Name':last_name,'Full Name':fullname,
            'School Name':school,'School Code':school_code,
            'Weight Class':weight_class,'Wins':wins,'Losses':losses,'Matches':matches})

In [ ]:
# Remake wrestlers df
# Note: union of winner/loser full names is set of all wrestlers in dataset
wrestlers = set(MATCHES['Winner Full Name']) | set(MATCHES['Loser Full Name'])
wrestlers = [x for x in wrestlers if x==x] # remove nan, convert to list
wrestler_data = [infoscrape(wrestler,MATCHES) for wrestler in wrestlers]
WRESTLERS = pd.DataFrame(wrestler_data)

In [ ]:
WRESTLERS.shape

In [ ]:
# Function to create train and test data split by date of wrestling events
# Note: research paper trained on one weight class and tested on all the rest.
# Why this is a big separate function: have to remake wrestlers dataframe from filtered matches dataframe

def train_test_split(match_data, wrestler_data=None, split_method='date',
                    earliest=None, latest=None, train_size=0.75):
    '''train_test_split creates train and test data using given match data.
    Can split by date range for train set or desired train data size (default is date).
    Train_size is between 0 and 1. earliest/latest are integer dates in format YYYYMMDD.
    Returns dict of match_train, match_test, wrestler_train, wrestler_test.'''
    
    event_dates = match_data["Event Date"]
    
    # Default dates
    if earliest is None:
        earliest = min(event_dates)
    if latest is None:
        latest = max(event_dates)
    
    # Handle input exceptions        
    if latest > max(event_dates):
        raise ValueError('Invalid indexing: latest ({}) cannot be after most recent event ({})'\
                         .format(latest,max(event_dates)))
    if earliest >= latest:
        raise ValueError('Invalid indexing: earliest ({}) must be less than latest ({})'\
                         .format(earliest,latest))
        
    # Train-Test Split
    
    if split_method == 'size': # split by train_size
        
        indices = match_data.index.values
        n = len(indices)
        train_end = int(n*train_size)
        train_id = range(0,train_end)
        test_id = range(train_end,n)
        match_train = match_data.iloc[train_id,:]
        match_test = match_data.iloc[test_id,:]
        
    if split_method == 'date': # split by date range
        
        date_range = range(earliest,latest+1)
        train_bool = [date in date_range for date in event_dates]
        test_bool = [not index for index in train_bool]
        match_train = match_data.loc[train_bool]
        match_test = match_data.loc[test_bool]
        
        
    # Name wrestlers to train or test sets
    wrestler_names_train = set(match_train['Winner Full Name']) | set(match_train['Loser Full Name'])
    wrestler_names_train = [x for x in wrestler_names_train if x==x] # remove nan, convert to list
    
    # Not sure if making wrestler test set like this makes total sense but I'll do it for now
    # Maybe because of cumulative stats, wrestler test set is always up-to-date full wrestler data?
    wrestler_names_test = set(match_test['Winner Full Name']) | set(match_test['Loser Full Name'])
    wrestler_names_test = [x for x in wrestler_names_test if x==x] # remove nan, convert to list

    # Call infoscrape to construct wrestler dataframes
    wrestler_train = [infoscrape(wrestler,match_train) for wrestler in wrestler_names_train]
    wrestler_train = pd.DataFrame(wrestler_train)
    wrestler_test = [infoscrape(wrestler,match_test) for wrestler in wrestler_names_test]
    wrestler_test = pd.DataFrame(wrestler_test)
    
    # Store train/test splits in dict
    train_test_dict = {"match_train":match_train,"match_test":match_test,
                      "wrestler_train":wrestler_train,"wrestler_test":wrestler_test}
    
    return(train_test_dict)

In [ ]:
def closest(arr, K): 
    '''returns the item in arr closest to the value K'''
    idx = (np.abs(arr - K)).argmin() 
    return(arr[idx])

In [ ]:
def elo_matchups(match_train):
    
    train_matchups = list()
    
    for idx, row in match_train.iterrows():
        w1 = row['Winner Full Name']
        w2 = row['Loser Full Name']
        
        # nan entry -> just have wrestler go against himself for now (should result in no winner)
        # no option for both nan, but that is a datapoint I don't even want
        if w1!=w1:
            w1 = w2
        elif w2!=w2:
            w2 = w1
            
        train_matchups.append((
            w1,w2
        ))
        
    return(train_matchups)

In [ ]:
def elote_func(a, b): # Currenly trivial since winner is already known to be first entry, a
    return True

In [ ]:
def match_recorder(matchup,wrestler_train,school_competitor_dict,initial=1000):
    '''Records a given matchup, implementing cold start ratings if needed, and updates school_competitor_dict.'''
    
    # Extract dict keys
    winner = matchup[0]
    loser = matchup[1]
    observed_schools = school_competitor_dict.keys()
    
    # Schools
    winner_bool = wrestler_train["Full Name"]==winner
    winner_school = wrestler_train.loc[winner_bool]["School Name"].values[0]
    loser_bool = wrestler_train["Full Name"]==loser
    loser_school = wrestler_train.loc[loser_bool]["School Name"].values[0]
    
    # Cold-start cases: Winner
    if winner_school not in observed_schools: # Wrestler receives static initial rating, begins their school's data
        school_competitor_dict[winner_school] = {}
        school_competitor_dict[winner_school][winner] = EloCompetitor(initial_rating=initial)
        
    else: # Wrestler has school data
        school_wrestlers = school_competitor_dict[winner_school].keys()
        
        if winner not in school_wrestlers: # Wrestler receives dynamic initial rating, mean of teammates ratings
            team_ratings = [competitor.rating for competitor in list(school_competitor_dict[winner_school].values())]
            winner_rating = np.mean(team_ratings)
            school_competitor_dict[winner_school][winner] = EloCompetitor(initial_rating=winner_rating)
            
    # Cold-start cases: Loser
    if loser_school not in observed_schools: # Wrestler receives static initial rating, begins their school's data
        school_competitor_dict[loser_school] = {}
        school_competitor_dict[loser_school][loser] = EloCompetitor(initial_rating=initial)
        
    else: # Wrestler has school data
        school_wrestlers = school_competitor_dict[loser_school].keys()
        
        if loser not in school_wrestlers: # Wrestler receives dynamic initial rating, mean of teammates ratings
            team_ratings = [competitor.rating for competitor in list(school_competitor_dict[loser_school].values())]
            loser_rating = np.mean(team_ratings)
            school_competitor_dict[loser_school][loser] = EloCompetitor(initial_rating=loser_rating)
            
    # Record match result
    school_competitor_dict[winner_school][winner].beat(school_competitor_dict[loser_school][loser])

In [ ]:
# Elo support provided via elote package by Will McGinnis @ wdm0006 on GitHub
# Remember to make this input a generic alg_args dict when that gets changed

def elo_rank(match_train,wrestler_train,initial=1000,K=225):
    '''elo_rank takes in match data along with elo tuning params and returns an ordered ranking of wrestlers
    along with the saved state of the elo arena'''
    
    
    # Form train matchups. For this, winner will always be first listed in each matchup.
    train_matchups = elo_matchups(match_train)
    
    # Create arena and process train matches
    arena = LambdaArena(elote_func)
    arena.set_competitor_class_var('_k_factor',K)
    school_competitor_dict = {} # dict of dicts of dicts (outer_dict -> school -> wrestler -> EloCompetitor Object)
    
    for matchup in train_matchups:
        match_recorder(matchup,wrestler_train,school_competitor_dict,initial)
        
    lb = [
        {'competitor':item[0],'rating':item[1].rating}
        for d in list(school_competitor_dict.values())
        for item in d.items()
    ]

    lb = sorted(lb, reverse=True, key=lambda x: x.get('rating'))
    
    # Return rankings and arena, post-train matches
    saved_state = arena.export_state()
    return({"Rankings":lb,
            "Saved Arena":saved_state})

In [ ]:
# Make some potentially useful elo ranking dictionaries

def elo_rank_dicts(raw_rankings):
    
    wrestlers_with_rankings = {} # keys are wrestlers, values are (rankings,ratings)
    rankings_with_wrestlers = {} # keys are rankings, (wrestlers,ratings) are values

    for index, comp_dict in enumerate(raw_rankings, start=1):
        rank = index
        wrestler,rating = comp_dict.values()
        rating = round(rating,3)
        wrestlers_with_rankings[wrestler] = (rank,rating)
        rankings_with_wrestlers[rank] = (wrestler,rating)
        
    return({"Ranked Wrestlers":wrestlers_with_rankings,
            "Named Rankings":rankings_with_wrestlers})

In [ ]:
def elo_pred(test_matchups,wrestler_train,wrestler_test,match_train,match_test): # alg_args please
    '''elo_pred predicts the winner of a bout to be the wrestler with the higher elo rating. Performs 
    predictions on all test matchups at once.'''
    
    
    # Raw rankings
    elo_rank_output = elo_rank(match_train,wrestler_train)
    elo_rankings = elo_rank_output["Rankings"]
    
    # Nicer dicts
    nice_dicts = elo_rank_dicts(elo_rankings)
    ranked_wrestlers = nice_dicts["Ranked Wrestlers"]
    named_rankings = nice_dicts["Named Rankings"]
    
    ## initial_state arg for arena call doesn't seem to work, commented out dependent code and implemented temp solution
    
    # Extract,implement arena info 
    #saved_state = elo_rank_output["Saved Arena"]
    #arena = LambdaArena(elote_func,initial_state=saved_state)
    
    # Simulate wrestlers' bout
    #winprob1 = arena.expected_score(wrestler1,wrestler2) # probability of wrestler1 beating wrestler2
    # Do we want a minimal difference to declare a decision?
    # Can reformat this into a more general confidence measure
    #prob_diff = abs(2*(winprob1 - 0.5))

    
    #if winprob1 > 0.5:
    #    return({"Winner":wrestler1,"Confidence":prob_diff})
    #elif winprob1 == 0.5:
    #    return({"Winner":None,"Confidence":prob_diff}) # TODO: Track these in validation
    #else:
    #    return({"Winner":wrestler2,"Confidence":prob_diff})
    
    
    # Temp solution
    pred_outputs = [] # list of dicts, each with two keys, 'Winner' and 'Confidence'
    default_rating = np.median([pair[1] for pair in ranked_wrestlers.values()]) # use median rating for default guess
    
    for matchup in test_matchups:
        
        output_dict = {}
        
        wrestler1 = matchup[0]
        wrestler2 = matchup[1]
    
        if wrestler1 not in ranked_wrestlers.keys():
            rating1 = default_rating
        else:
            rating1 = ranked_wrestlers[wrestler1][1]

        if wrestler2 not in ranked_wrestlers.keys():
            rating2 = default_rating
        else:
            rating2 = ranked_wrestlers[wrestler2][1]
            
        rating_diff = abs(rating1-rating2) # make this a comparable confidence format; see Win Percentage Pred function
        output_dict['Confidence'] = rating_diff
               
        
        if rating1 > rating2: # Wrestler 1 wins
            output_dict['Winner'] = wrestler1
        elif rating1==rating2:
            output_dict['Winner'] = None
        else: # Wrestler 2 wins
            output_dict['Winner'] = wrestler2
            
        pred_outputs.append(output_dict)
            
    return(pred_outputs)
    

In [ ]:
# Give pred algorithms attributes so that the alogorithm tester can test them differently (for efficiency)
# Can also probably just work test_method into algorithm args
setattr(elo_pred,'test_method','batch') # batch because elo_pred can rank everyone and predict in one go

In [ ]:
def matchmaker(match_test):
    '''matchmaker takes in match test data and returns a list of 
    the associated wrestler matchup pairs'''
    
    
    test_matchups = []
    
    for i in range(0,match_test.shape[0]):
        match = match_test.iloc[i]
        w1 = match["Winner Full Name"]
        w2 = match["Loser Full Name"]
        
        # nan entry -> just have wrestler go against himself for now (should result in no winner)
        # no option for both nan, but that is a datapoint I don't even want
        if w1!=w1:
            w1 = w2
        elif w2!=w2:
            w2 = w1
        
        test_matchups.append((w1,w2)) # winner is first entry in each matchup tuple for testing
        
    return(test_matchups)

In [ ]:
def test_algorithm(algorithm,match_train,match_test,wrestler_train,wrestler_test):
    '''test_algorithm implements a given algorithm using given wrestler and match 
    train/test data and returns prediction accuracy'''
    
    # Extract matchups from test matches
    test_matchups = matchmaker(match_test)
    
    # Extract test_method attribute from algorithm
    test_method = algorithm.test_method

    # True and predicted winners
    true_winners = match_test["Winner Full Name"]
    
    if test_method == 'batch': # Elo test
        pred_output = algorithm(test_matchups,wrestler_train,wrestler_test,match_train,match_test)
    else: # WP test
        pred_output = [algorithm(W1,W2,wrestler_train,wrestler_test,match_train,match_test) for W1,W2 in test_matchups]
        
    pred_winners = [output["Winner"] for output in pred_output]
    pred_confidences = [output["Confidence"] for output in pred_output] # Needs normalization for comparison b/t algs
    
    # Calculate prediction accuracy, save incorrect pred info
    no_decision_bool = [pred is None for pred in pred_winners]
    no_decisions = sum(no_decision_bool)
    no_decision_preds = match_test.loc[no_decision_bool,:]
    pred_results = true_winners == pred_winners
    incorrect_preds = match_test.loc[true_winners != pred_winners,:]
    n = len(pred_results)
    correct = sum(pred_results)
    incorrect = n - correct
    pred_accuracy = pred_results.mean()
    
    return({"Accuracy":pred_accuracy,"NumCorrect":correct,"NumIncorrect":incorrect,"NumUndecided":no_decisions,"N":n,
           "WrongPreds":incorrect_preds,"UndecidedPreds":no_decision_preds,"PredConfidences":pred_confidences})

In [ ]:
# Test algorithms; various training sizes. Takes 10-15 minutes cycling through each train_size
train_sizes = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

elo_results = {}

for size in train_sizes:
    
    # latest = int(MATCHES['Event Date'].quantile(q=size))
    train_test_dict = train_test_split(MATCHES,split_method='size',train_size=size)
    match_train = train_test_dict["match_train"]
    wrestler_train = train_test_dict["wrestler_train"]
    match_test = train_test_dict["match_test"]
    wrestler_test = train_test_dict["wrestler_test"]
    
    elo_pred_results = test_algorithm(elo_pred,match_train,match_test,wrestler_train,wrestler_test)
    
    elo_results[size] = elo_pred_results

In [ ]:
# Extract accuracy values
elo_accuracy_dict = {}

for size in train_sizes:
    elo_accuracy_dict[size] = elo_results[size]['Accuracy']

In [ ]:
max(elo_accuracy_dict.values())

In [ ]:
# Plot accuracy results: Dynamic Elo
plt.bar(range(len(elo_accuracy_dict)), list(elo_accuracy_dict.values()), align='center')
plt.xticks(range(len(elo_accuracy_dict)), list(elo_accuracy_dict.keys()))
plt.title('Prediction Accuracy')
plt.xlabel('Train Set Size')
plt.ylabel('Accuracy')

# Save fig
plt.savefig('./Plots/Elo/dynamic_pred_accuracy.png')

plt.show()

# Observation: Elo performs worse earlier, better later
# Grain of salt: we don't even have the full season data, just late season

In [ ]:
# Narrow diagnostics to best (0.9) train size results for now
# Investigate erroneous predictions
elo_pred_results = elo_results[0.9]

In [ ]:
elo_pred_results['NumUndecided']

In [ ]:
elo_mistakes = elo_pred_results['WrongPreds']

In [ ]:
print("Dynamic Elo Mistakes:",elo_mistakes.shape[0])

In [ ]:
elo_mistakes.groupby('Event Name').size().reset_index(name='counts') # Mistakes by event name, Elo

In [ ]:
# Show distribution of weight classes among wrestlers in incorrect pred cases
# Misleading: should scale by num wrestler/matches in weight class
# Note: make this a bar chart instead if possible
elo_pred_results['WrongPreds'].hist(column="Weight Class")
plt.xlabel("Weight Class")
plt.ylabel("Number of Incorrect Predictions")
plt.title("Incorrect Elo Preds by Weight Class")

# Save fig
plt.savefig('./Plots/Elo/dynamic_incorrect_preds_weight_class.png')

In [ ]:
# Show distribution of weight classes among wrestlers in incorrect pred cases
# Misleading: should scale by num wrestler/matches in weight class
# Note: make this a bar chart instead if possible
elo_pred_results['WrongPreds'].hist(column="Event Date",bins=25)
plt.xlabel("Event Date")
plt.ylabel("Number of Incorrect Predictions")
plt.title("Incorrect Dynamic Elo Preds by Event Date")
# fix x axis scale

In [ ]:
# Show distribution of number of matches among wrestlers
WRESTLERS.hist(column="Matches")
plt.xlabel("Number of Matches")
plt.ylabel("Number of Wrestlers")
plt.title("Distribution of Wrestlers by Number of Matches")

# Save fig
plt.savefig('./Plots/EDA/wrestler_match_dist.png')

# Vast majority of wrestlers have less than 15 matches

In [ ]:
# Show distribution of matches by weight class
wrestlers_by_weight = WRESTLERS.groupby('Weight Class')
wrestlers_by_weight.sum().plot.bar(y="Matches")
plt.title("Distribution of Matches by Weight Class")

# Save fig
plt.savefig('./Plots/EDA/matches_by_weight_class.png')

# Fairly balanced weight classes; flattened bell curve

In [ ]:
# Check count of matches by victory type 
# There are nan entries for Victory Type (L)
#matches_by_wintype = MATCHES.groupby('Victory Type (L)')
#matches_by_wintype.describe()['Match ID'].plot.bar(y='count')
#plt.title("Distribution of Matches by Victory Type")

# Save fig
#plt.savefig('./Plots/matches_by_win_type.png')

# Practically all victory types are fall or decision